In [1]:
import glob
import time

import tensorflow as tf
import cv2
import keras
import numpy as np

import dlib

In [2]:
def predict(image, height=224, width=224):
    im = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    im = im / 255
    im = cv2.resize(im, (height, width))
    im = im.reshape((1,) + im.shape)

    pred = model.predict(im)

    mask = pred.reshape((224, 224))

    return mask


def transfer(image, mask):
    mask[mask > 0.5] = 255
    mask[mask <= 0.5] = 0

    mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    mask_n = np.zeros_like(image)
    mask_n[:, :, 2] = mask

    alpha = 0.8
    beta = (1.0 - alpha)
    dst = cv2.addWeighted(image, alpha, mask_n, beta, 0.0)

    return dst


def getHead(hog_face_detector, image):
    faces_hog = hog_face_detector(image, 1)

    heads = []
    
    for face in faces_hog:
        
        head = dict()
        
        head["left"] = max(face.left() - 300, 0)
        head["top"] = max(face.top() - 300, 0)
        head["right"] = min(face.right() + 300, image.shape[0])
        head["bottom"] = min(face.bottom() + 300, image.shape[1])
        
        heads.append(head)

    return heads

In [3]:
#Loading the saved model
model = keras.models.load_model('checkpoints/model/checkpoint.hdf5')

hog_face_detector = dlib.get_frontal_face_detector()

In [ ]:
for path_image in glob.glob('test/images/*'):
    image = cv2.imread(path_image)
    heads = getHead(hog_face_detector, image)

    for head in heads:
        img = image[head["top"]:head["bottom"], head["left"]:head["right"]]
        img = cv2.resize(img, (350,350))

        mask = np.zeros_like(img)
        mask = predict(img)
        
        mask[mask > 0.5] = 255
        mask[mask <= 0.5] = 0
        

        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
        mask = mask.reshape((img.shape[1], img.shape[0], 1))
        mask = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
        mask_n = np.zeros_like(mask)
        mask_n[:] = 200,0,150
        mask_n = cv2.bitwise_and(mask, mask_n)
        mask_nn = np.zeros_like(img)
        mask_nn[:, :] = mask_n
        # cv2.imshow('After Hair Colouring', mask_nn)
        # cv2.waitKey(0)
        # mask_n[:, :, 2] = mask

        alpha = 0.7
        beta = (1.0 - alpha)
        dst = cv2.addWeighted(img, alpha, mask_nn, beta,0, mask_nn)

        collage = np.hstack((img, dst))
        cv2.imshow('After Hair Colouring', collage)
        cv2.waitKey(0)
cv2.destroyAllWindows()

## Working on Single Images

In [13]:
img1  = cv2.imread('./assets/img9.jpg')
heads = getHead(hog_face_detector, img1)
for head in heads:
    img = img1[head["top"]:head["bottom"], head["left"]:head["right"]]
    img = cv2.resize(img, (350,350))

    mask = np.zeros_like(img)
    mask = predict(img)
        
    mask[mask > 0.5] = 255
    mask[mask <= 0.5] = 0
        

    mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
    mask = mask.reshape((img.shape[1], img.shape[0], 1))
    mask = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
    mask_img = np.zeros_like(img)
    mask_img[:,:] = mask
    mask_n = np.zeros_like(mask)
    mask_n[:] = 200,150,0
    mask_n = cv2.bitwise_and(mask, mask_n)
    mask_nn = np.zeros_like(img)
    mask_nn[:, :] = mask_n

    alpha = 0.8
    beta = (1.0 - alpha)
    dst = cv2.addWeighted(img, alpha, mask_nn, beta,0, mask_nn)

    collage = np.hstack((img, mask_img, dst))
    cv2.imshow('After Hair Colouring', collage)
    cv2.waitKey(0)
cv2.destroyAllWindows()

## Interactive Window

In [17]:
def empty(a):
    pass

cv2.namedWindow('HairColouring')
cv2.resizeWindow('HairColouring', 640, 240)
cv2.createTrackbar("Blue",'HairColouring',0,255,empty)
cv2.createTrackbar("Green",'HairColouring',0,255,empty)
cv2.createTrackbar("Red",'HairColouring',0,255,empty)


while True:
    img1  = cv2.imread('./assets/img9.jpg')
    heads = getHead(hog_face_detector, img1)
    for head in heads:
        img = img1[head["top"]:head["bottom"], head["left"]:head["right"]]
        img = cv2.resize(img, (350,350))

        mask = np.zeros_like(img)
        mask = predict(img)
            
        mask[mask > 0.5] = 255
        mask[mask <= 0.5] = 0
            

        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
        mask = mask.reshape((img.shape[1], img.shape[0], 1))
        mask = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
        mask_n = np.zeros_like(mask)
        b = cv2.getTrackbarPos('Blue','HairColouring')
        g = cv2.getTrackbarPos('Green','HairColouring')
        r = cv2.getTrackbarPos('Red','HairColouring')
        mask_n[:] = b,g,r
        mask_n = cv2.bitwise_and(mask, mask_n)
        mask_nn = np.zeros_like(img)
        mask_nn[:, :] = mask_n

        alpha = 0.8
        beta = (1.0 - alpha)
        dst = cv2.addWeighted(img, alpha, mask_nn, beta,20, mask_nn)

        collage = np.hstack((img, dst))
        cv2.imshow('HairColouring', collage)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()